In [23]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 10.0)
import os
from collections import Counter

import json
import math
import os

from keras.backend import clear_session
import matplotlib.pyplot as plt
import numpy as np

from abyss_deep_learning.datasets.coco import ImageClassificationDataset
from abyss_deep_learning.datasets.translators import CocoCaptionTranslator
from abyss_deep_learning.keras.classification import caption_map_gen, onehot_gen, hamming_loss
from abyss_deep_learning.keras.utils import lambda_gen, batching_gen
from abyss_deep_learning.keras.models import ImageClassifier
from abyss_deep_learning.utils import balanced_set
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, TensorBoard, EarlyStopping


In [2]:
from abyss_deep_learning.datasets.coco import ImageClassificationDataset
from abyss_deep_learning.datasets.translators import AnnotationTranslator

In [3]:
DATA_DIR='/mnt/rhino/datasets/abyss-internal/coco2017'
dataType='val2017'
annFile='{}/annotations/instances_{}.json'.format(DATA_DIR,dataType)

In [4]:
def cat_strings(self, cat_ids):
    return self.coco.loadCats(cat_ids)

In [5]:
class coco_2017_translator(AnnotationTranslator):
    '''base class to transform annotations'''
    def __init__(self):
        pass
    def filter(self, annotation):
        '''Whether or not to use a annotation'''
        return True
    def translate(self, annotation):
        '''Transform the annotation in to a list of captions'''
        return [annotation['category_id']]

ds = ImageClassificationDataset(
    annFile,
    translator=coco_2017_translator(),
    cached=False)

# val_ds = ImageClassificationDataset(
#     '{}/annotations/instances_{}.json'.format(DATA_DIR,'val2017'),
#     translator=coco_2017_translator(),
#     cached=False)
# ds.caption_map

loading annotations into memory...
Done (t=3.11s)
creating index...
index created!


In [6]:
# Find a balanced set
def balanced_set(ds, ignore_captions=None):
    ignore_captions = ignore_captions or []
    captions = ds.captions
    ds._calc_class_stats()

    smallest_caption = min(ds.stats['images_per_class'], key=ds.stats['images_per_class'].get)
    smallest_caption_value = ds.stats['images_per_class'][smallest_caption]

    captions_by_frequency = sorted(ds.stats['images_per_class'], key=lambda x: ds.stats['images_per_class'][x], reverse=False)
    
    # Count how many captions are in each image
    captions_in_image = {
        image_id: list(set([a['category_id'] for a in ds.coco.loadAnns(ds.coco.getAnnIds(imgIds=[image_id]))]))
        for image_id in ds.coco.getImgIds()}
    balanced = []
    out = {caption: [] for caption in captions}

    images_in_caption = {
        caption: ds.coco.getImgIds(catIds=[caption])
        for caption in captions}
    for images in images_in_caption.values():
        np.random.shuffle(images)

    sorted_images_in_caption = {
        caption: sorted(images_in_caption[caption], key=lambda i: np.sum(
            [x * 1 / ds.stats['class_weights'][caption] for x in captions_in_image[i]]))
        for caption in captions
    }

    def add_to_counts(image_id):
        # Increment counts for all captions in image
        for caption in captions_in_image[image_id]:
            out[caption].append(image_id)
        # Remove image_id from all images_in_caption
        for images in images_in_caption.values():
            if image_id in images:
                images.pop(images.index(image_id))
    
    while any([len(out[caption]) < smallest_caption_value for caption in captions]):
        least = min(out.items(), key=lambda x: len(x[1]))
        image_id = sorted_images_in_caption[least[0]].pop()
        add_to_counts(image_id)
        
    # print("balanced images in caption")
    # print({k: len(v) for k, v in out.items()})
    out = set([j
           for i in out.values()
          for j in i])

    return out

In [24]:
balanced_set(ds)

KeyError: '1'

In [20]:
d = {
     img_id:
     set(
         [a['category_id'] for a in ds.coco.loadAnns(ds.coco.getAnnIds(imgIds=[img_id]))]
     ) 
     for img_id in balanced_set(ds)}


In [21]:
d

{14038: {44, 62, 63, 64, 65, 67, 72, 77, 78, 82, 84},
 16451: {1, 27, 28, 31, 42, 62},
 17899: {1, 16, 47, 50, 58, 61, 62, 63, 67},
 20553: {28, 44, 47, 62, 67, 86, 88},
 25096: {1, 24, 41, 49, 62, 67},
 26690: {1, 41, 82},
 35279: {1, 31, 43, 44, 73, 77, 84},
 35682: {1, 31, 47, 60, 61, 62, 67},
 39405: {1, 15, 27, 33, 37, 43, 62},
 39484: {1, 3, 5, 10, 62, 67},
 42296: {23, 37},
 54593: {1, 3, 4, 27, 37, 39, 40, 44, 62},
 55528: {1, 63, 75, 84, 85, 90},
 59386: {3, 24, 25},
 60507: {1, 37, 39, 40, 62, 77},
 60899: {1, 19, 62, 84},
 61584: {1, 19, 62, 67},
 67616: {1, 2, 3, 8, 11, 14, 62, 64, 67, 86},
 74209: {44, 47, 51, 52, 53, 55, 62, 67, 79, 81, 82, 85},
 74256: {1, 6, 7, 27, 31, 77},
 74646: {1, 38, 47, 62},
 76547: {1, 2, 7, 10, 15, 27, 32, 62, 67, 73},
 78170: {1, 7, 31, 44, 62, 77},
 84241: {1, 44, 48, 50, 51, 54, 56, 67, 79, 81},
 84674: {1, 60, 63, 72, 84, 85},
 91500: {1, 44, 47, 51, 59, 62, 67, 75, 81},
 91779: {47, 51, 54, 58, 62, 67},
 92053: {1, 47, 48, 49, 51, 54, 60, 

In [22]:
out = []
out += [x for val in d.values() for x in list(val)]
Counter(out)

Counter({1: 136,
         2: 10,
         3: 22,
         4: 6,
         5: 8,
         6: 8,
         7: 8,
         8: 11,
         9: 6,
         10: 8,
         11: 7,
         13: 8,
         14: 6,
         15: 18,
         16: 8,
         17: 9,
         18: 10,
         19: 7,
         20: 7,
         21: 7,
         22: 7,
         23: 8,
         24: 7,
         25: 8,
         27: 34,
         28: 17,
         31: 40,
         32: 8,
         33: 8,
         34: 8,
         35: 5,
         36: 4,
         37: 13,
         38: 6,
         39: 7,
         40: 5,
         41: 7,
         42: 6,
         43: 8,
         44: 64,
         46: 8,
         47: 51,
         48: 20,
         49: 20,
         50: 16,
         51: 34,
         52: 9,
         53: 6,
         54: 11,
         55: 6,
         56: 8,
         57: 9,
         58: 7,
         59: 7,
         60: 7,
         61: 10,
         62: 93,
         63: 20,
         64: 27,
         65: 6,
         67: 65,
         7

In [51]:
di=ds.data_ids[0]
di

532481

In [52]:
ds.coco.getImgIds()

[532481,
 66231,
 458755,
 245764,
 385029,
 343706,
 311303,
 334977,
 393226,
 532493,
 475150,
 458768,
 62808,
 438269,
 8211,
 327701,
 188439,
 376856,
 241668,
 434204,
 221213,
 409630,
 372349,
 24610,
 190676,
 458790,
 213033,
 466986,
 213035,
 368684,
 314034,
 319534,
 122927,
 188465,
 270386,
 172083,
 434230,
 475191,
 499768,
 76468,
 131131,
 275727,
 499775,
 254016,
 417632,
 393282,
 106563,
 376900,
 120572,
 507975,
 180296,
 516173,
 241677,
 243626,
 122962,
 262227,
 417876,
 8277,
 555705,
 442456,
 122969,
 324614,
 528399,
 491613,
 213086,
 532575,
 90208,
 221281,
 311394,
 490171,
 327780,
 522940,
 221291,
 29984,
 163951,
 442480,
 78565,
 360564,
 49269,
 16502,
 32887,
 434297,
 531135,
 450686,
 286849,
 32901,
 540932,
 338625,
 468332,
 270474,
 139,
 424642,
 565391,
 196754,
 581781,
 196759,
 245915,
 180383,
 491683,
 82085,
 385190,
 524456,
 90284,
 32941,
 188592,
 278705,
 460147,
 385205,
 426166,
 114871,
 537270,
 128372,
 159311,
 286

In [53]:
images_in_caption = {
        str(caption): ds.coco.getImgIds(catIds=[caption])
        for caption in ds.captions}
images_in_caption

{'1': [532481,
  458755,
  385029,
  311303,
  393226,
  532493,
  8211,
  327701,
  188439,
  434204,
  401446,
  213033,
  466986,
  213035,
  319534,
  516143,
  32817,
  532530,
  188465,
  434230,
  475191,
  254016,
  131138,
  16451,
  81988,
  376900,
  507975,
  434247,
  204871,
  516173,
  122962,
  98392,
  442456,
  32861,
  213086,
  442463,
  311392,
  311394,
  49259,
  221291,
  442480,
  368752,
  319607,
  417911,
  32887,
  450686,
  32901,
  270474,
  139,
  180383,
  82085,
  385190,
  524456,
  90284,
  32941,
  188592,
  229553,
  278705,
  213171,
  336053,
  549055,
  114884,
  508101,
  65736,
  303305,
  344268,
  532690,
  360661,
  16598,
  73946,
  114907,
  426203,
  278749,
  516318,
  229601,
  295138,
  467176,
  319721,
  352491,
  33005,
  458992,
  475387,
  393469,
  57597,
  426241,
  540932,
  106757,
  65798,
  180487,
  213255,
  147725,
  418062,
  147729,
  377113,
  229659,
  147740,
  426268,
  434459,
  205105,
  565563,
  278848,
  36896

In [ ]:
ds.coco.loadImgs()

In [54]:
captions_in_image = {
        str(image_id): set([a['category_id'] for a in ds.coco.loadAnns(ds.coco.getAnnIds(imgIds=[image_id]))])
        for image_id in ds.coco.getImgIds()}
captions_in_image

{'314182': {51, 56, 57},
 '198928': {1, 3, 6, 10},
 '414170': {1, 27, 35},
 '369757': {58, 59},
 '8532': {1, 32},
 '155341': {1, 3, 8},
 '32901': {1, 32, 44, 46, 47, 62, 67},
 '62692': {1, 43},
 '32038': {1, 44, 59, 67},
 '17207': {1, 3, 4, 6},
 '164602': {1, 81, 90},
 '203580': {15, 28, 67},
 '66523': {65},
 '363188': {1, 3, 27, 44, 77},
 '46497': {1, 9},
 '327617': {1, 43},
 '423944': {32},
 '288862': {1, 41},
 '451308': {33, 84},
 '5600': {50, 51},
 '178028': {44, 46, 47, 81, 89},
 '365095': {1, 37},
 '327769': {17, 44, 47, 70, 81, 90},
 '17627': {1, 3, 6},
 '493613': {41},
 '143931': {1, 6},
 '532690': {1, 63, 75},
 '498807': {1, 42},
 '478136': {60, 61, 67},
 '22371': {1, 32, 73, 84},
 '205514': {62, 63, 64, 67, 72, 84, 86},
 '474854': {1, 52, 62, 67},
 '530820': {1, 43},
 '215723': {1, 3, 8, 10, 28},
 '382111': {1, 3, 8, 19},
 '550939': set(),
 '121417': {1, 15, 28, 31, 77},
 '517832': {18, 62},
 '314177': {1, 70},
 '483999': {1, 28, 47, 48, 49, 50, 62, 67},
 '132703': {1, 41},
 

In [55]:
img = ds.coco.loadImgs(ds.coco.getImgIds([di]))
print(ds.coco.loadCats([1]))
I = io.imread(img['coco_url'])
plt.imshow(I)

[{'id': 1, 'supercategory': 'person', 'name': 'person'}]


TypeError: list indices must be integers or slices, not str

In [ ]:
out = {caption: [] for caption in ds.captions}
out

In [ ]:
ds._calc_class_stats()

In [ ]:
sorted(ds.stats['images_per_class'], key=lambda x: ds.stats['images_per_class'][x], reverse=False)

In [ ]:
ds.stats['images_per_class']

In [ ]:
from collections import Counter

In [ ]:
ds.coco.cats

In [ ]:
ds.stats['class_weights']

In [ ]:
images_in_caption

In [ ]:
ds.stats['class_weights']

In [ ]:
caption = 1
[np.sum([x*1/ds.stats['class_weights'][caption] for x in captions_in_image[str(i)]]) for i in images_in_caption[str(caption)]]

In [ ]:
captions = ds.captions
sorted_images_in_caption = {
        caption: sorted(images_in_caption[str(caption)], key=lambda i: np.sum([x*1/ds.stats['class_weights'][caption] for x in captions_in_image[str(i)]]))
        for caption in captions
    }

In [ ]:
sorted_images_in_caption


In [ ]:
ds.coco.getAnnIds(imgIds=list(balanced_set(ds)))

In [56]:
d = {
     img_id:
     set(
         [a['category_id'] for a in ds.coco.loadAnns(ds.coco.getAnnIds(imgIds=[img_id]))]
     ) 
     for img_id in balanced_set(ds)}

KeyError: '1'

In [ ]:
out = []
out += [x for val in d.values() for x in list(val)]
Counter(out)

In [ ]:
ds.coco.loadAnns(ds.coco.getAnnIds(imgIds=[di]))

In [ ]:
ds.coco.loadCats(ds.coco.getCatIds())

In [ ]:
ds.coco.loadCats(ds.coco.getCatIds())

In [ ]:
caption_map = {d['name']: d['id'] for d in ds.coco.loadCats(ds.coco.getCatIds())}
print(caption_map)
caption_map_r = {str(d['id']): d['name'] for d in ds.coco.loadCats(ds.coco.getCatIds())}
print(caption_map_r)

In [ ]:
ds.coco.getImgIds(catIds=[1])

In [ ]:
ds.coco.anns.

In [ ]:
caption_map_r['3']

In [ ]:
def pipeline(gen):
    """ Pipeline consists of lambda expression mapping x -> x, and y to become a set of comma separated captions
        i.e.  {'ED,IP'} -> {'ED', 'IP'}
        caption_map_gen converts caption labels to numeric integers
        onehot_gen converts numeric integers to a vector of 1's and 0's where 1 is a given label
    """
    return onehot_gen(
        gen,
        len(ds.coco.getCatIds())
    )

def create_new_model(ds):
    clear_session()
    model = ImageClassifier(
        backbone='xception', input_shape=(None, None, 3), classes=len(ds.coco.getCatIds()),
        init_lr=1e-5, init_weights='imagenet',
#         trainable=True, loss='binary_crossentropy', output_activation='sigmoid',
#         metrics=['accuracy']
    )
    return model

In [ ]:
from sklearn.base import BaseEstimator
help(BaseEstimator)

In [ ]:
ds.coco.dataset['images'][0]

In [ ]:
next(ds.generator())[1]

In [ ]:
train_ds = ds

In [16]:
def setup_directory(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        

In [17]:
# def create_new_head(base_model, num_classes, caption_type, train_features=False, opt_params={}):
#     '''make sure base_model has include_top=False'''
#     from keras.layers import Dense, MaxPooling2D, Dropout, Flatten
#     from keras.models import Model
    
#     if not opt_params:
#         opt_params = {"optimizer": "Nadam"}
#     opt_params['loss'] = "categorical_crossentropy" if caption_type == "single" else "binary_crossentropy"
#     activation = "softmax" if caption_type == "single" else "sigmoid"
    
#     # add a global spatial average pooling layer
#     x = base_model.output
#     x = MaxPooling2D(pool_size=(2, 2))(x)
#     x = Dropout(0.25)(x)
#     x = Flatten()(x)
#     x = Dense(1024, activation='relu')(x)
#     x = Dense(1024, activation='relu')(x)
#     x = Dropout(0.5)(x)
#     predictions = Dense(num_classes, activation=activation, name='class_logits')(x)


#     # this is the model we will train
#     model = Model(inputs=base_model.input, outputs=predictions)
# #     model = multi_gpu_model(model, gpus=2)
#     # first: train only the top layers (which were randomly initialized)
#     # i.e. freeze all convolutional InceptionV3 layers
#     for layer in base_model.layers:
#         layer.trainable = train_features
#     # compile the model (should be done *after* setting layers to non-trainable)
#     model.compile(**opt_params, metrics=['accuracy'])
    
#     return model

In [31]:
model.model_

In [18]:
log_dir = "/home/jackson/coco"

In [19]:
best_path = os.path.join(log_dir, "models/best.{epoch:03d}-{val_loss:.4f}.h5")
setup_directory(os.path.dirname(best_path))

callbacks=[
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=9, cooldown=6, verbose=1),
        ModelCheckpoint(
            best_path, monitor='val_loss', verbose=1,
            save_best_only=True, save_weights_only=True, mode='auto', period=1),
        TensorBoard(
            log_dir=log_dir,
            histogram_freq=0,
            batch_size=batch_size,
            write_graph=False,
            write_grads=False,
            write_images=False),
#         EarlyStopping(
#             monitor='val_loss', min_delta=0.0, patience=40, verbose=1, mode='auto')
]


In [20]:
val_ds = ds
train_ds = ds

In [24]:
next(batching_gen(pipeline(train_ds.generator(shuffle_ids=True)), batch_size=batch_size))[0].shape

/home/jackson/.local/lib/python3.5/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function onehot_gen is deprecated; Use the new dataset translators interface.
  warnings.warn(msg, category=DeprecationWarning)


(2,)

In [44]:
#[0 if x < val else 1 with([x.name for x in model.model_.layers].index('block1_conv1')]) as val]

SyntaxError: invalid syntax (<ipython-input-44-40c76e99ba12>, line 1)

AttributeError: __exit__

In [21]:
batch_size = 2
#steps_per_epoch = math.floor(len(train_ds.coco.getImgIds()) / batch_size)
steps_per_epoch = 100

model = create_new_model(ds)
print("Break-even loss is", -np.log(1 / model.classes))
#model.save_on_epoch_end()
model.fit_generator(batching_gen(pipeline(train_ds.generator(shuffle_ids=True)), batch_size=batch_size),
                    validation_data=batching_gen(pipeline(val_ds.generator(endless=True)), batch_size=batch_size), 
                    validation_steps=100,
                    epochs=10, 
                    use_multiprocessing=True,
                    steps_per_epoch=steps_per_epoch,
                    callbacks=callbacks,
                    verbose=1)

Break-even loss is 4.382026634673881


/home/jackson/.local/lib/python3.5/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function onehot_gen is deprecated; Use the new dataset translators interface.
  warnings.warn(msg, category=DeprecationWarning)
/home/jackson/.local/lib/python3.5/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function onehot_gen is deprecated; Use the new dataset translators interface.
  warnings.warn(msg, category=DeprecationWarning)


Epoch 1/10


ValueError: Error when checking input: expected input_1 to have 4 dimensions, but got array with shape (2, 1)

In [ ]:
model.save("/home/jackson/coco")